# Exploration

In [52]:
import nltk
import pandas as pd
import re
import string
#from wordcloud import WordCloud
import matplotlib.pyplot as plt
import sklearn
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import  WordPunctTokenizer
from nltk.tokenize import  BlanklineTokenizer
from nltk.stem import WordNetLemmatizer
import stanfordnlp

In [2]:
nltk.download("stopwords")
stop_words = nltk.corpus.stopwords.words("english")
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Marco\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Marco\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Marco\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
df = pd.read_csv("data/labeled_data.csv", sep = ',').drop("Unnamed: 0", axis=1)
df.head()

,count,hate_speech,offensive_language,neither,class,tweet
0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [3]:
df.describe()

,count,hate_speech,offensive_language,neither,class
count,24783.000000,24783.000000,24783.000000,24783.000000,24783.000000
mean,3.243473,0.280515,2.413711,0.549247,1.110277
std,0.883060,0.631851,1.399459,1.113299,0.462089
min,3.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,0.000000,2.000000,0.000000,1.000000
50%,3.000000,0.000000,3.000000,0.000000,1.000000
75%,3.000000,0.000000,3.000000,0.000000,1.000000
max,9.000000,7.000000,9.000000,9.000000,2.000000


In [4]:
df.loc[20000]

count                                                                 3
hate_speech                                                           0
offensive_language                                                    3
neither                                                               0
class                                                                 1
tweet                 RT @shananigans__: 50 shades of shut your bitc...
Name: 20000, dtype: object

In [19]:
df["count"].value_counts()

3    22807
6     1571
4      211
9      167
7       27
Name: count, dtype: int64

In [20]:
df["hate_speech"].value_counts()

0    19790
1     3419
2     1251
3      287
4       21
5        7
6        5
7        3
Name: hate_speech, dtype: int64

In [26]:
df[df["hate_speech"] == 7]["tweet"][6171]

'@infidelpamelaLC I\'m going to blame the black man, since they always blame "whitey" I\'m an equal opportunity hater.'

In [27]:
df.loc[6171]

count                                                                 9
hate_speech                                                           7
offensive_language                                                    1
neither                                                               1
class                                                                 0
tweet                 @infidelpamelaLC I'm going to blame the black ...
Name: 6171, dtype: object

## FRASE TEST

In [6]:
text = df["tweet"][24]
text

'" got ya bitch tip toeing on my hardwood floors " &#128514; http://t.co/cOU2WQ5L4q'

## Tokenization

In [20]:
tt = TweetTokenizer()
wpt = WordPunctTokenizer()
blt = BlanklineTokenizer()

In [21]:
print(tt.tokenize(text.lower())) # Tweet
print(wpt.tokenize(text.lower())) # Wordpunct
print(blt.tokenize(text.lower())) # Blankline

['"', "don't", 'make', 'me', 'make', 'you', 'fall', 'in', 'live', 'with', 'a', 'nigga', 'like', 'meee', '...', '"', 'the', 'birds', '1', '&', '2', 'are', 'my', 'favorite', 'songs', 'by', 'weeknd']
['"', 'don', "'", 't', 'make', 'me', 'make', 'you', 'fall', 'in', 'live', 'with', 'a', 'nigga', 'like', 'meee', '..."', 'the', 'birds', '1', '&', 'amp', ';', '2', 'are', 'my', 'favorite', 'songs', 'by', 'weeknd']
['"don\'t make me make you fall in live with a nigga like meee..." the birds 1&amp;2 are my favorite songs by weeknd']


## Removing stopwords

In [27]:
tokens_text = tt.tokenize(text)
remove_sw = []
for token in tokens_text:
    if token.lower() not in stop_words:
         remove_sw.append(token)
print(remove_sw)

['"', 'make', 'make', 'fall', 'live', 'nigga', 'like', 'meee', '...', '"', 'birds', '1', '&', '2', 'favorite', 'songs', 'Weeknd']


## Remove numbers

Questa parte va inserita prima della rimozione delle stopwords

In [30]:
remove_num = re.sub(r'\d+', '', text)
print(remove_num)

"Don't make me make you fall in live with a nigga like meee..." The birds &amp; are my favorite songs by Weeknd


## Remove punctuations

Ovviamente pure sta parte va fatta prima

In [83]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [31]:
remove_punc = text.translate(str.maketrans('', '', string.punctuation))
print(remove_punc)

Dont make me make you fall in live with a nigga like meee The birds 1amp2 are my favorite songs by Weeknd


## Remove Extra spaces

Direi parecchio utile

In [32]:
remove_sp = re.sub(r'\s\s', '', text)
print(remove_sp)

"Don't make me make you fall in live with a nigga like meee..." The birds 1&amp;2 are my favorite songs by Weeknd


## WordCloud

In [ ]:
def plot_wordcloud(cnt, file_name="figure1.png"):
    wordcloud = WordCloud(max_font_size=50, max_words=100, background_color="white")
    wordcloud.generate_from_frequencies(dict(cnt))
    plt.figure()
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    # plt.show()
    plt.savefig(file_name)

## Stemming

In [ ]:
tokenized_text = WordPunctTokenizer().tokenize(text)

porter = PorterStemmer()
stemmed = [porter.stem(word) for word in tokenized_text]

## Stanford

In [37]:

# stanfordnlp.download('en')   # This downloads the English models. Run only once!
base = "C:\\Users\\Marco\\stanfordnlp_resources\\en_ewt_models\\"
config = {
    'processors': 'tokenize,pos,lemma,depparse', # Comma-separated list of processors to use
    'lang': 'en', # Language code for the language to build the Pipeline in
    'model_path': base + 'en_ewt_tokenizer.pt', 
    'pos_model_path': base + 'en_ewt_tagger.pt', 
    'pos_pretrain_path': base + 'en_ewt.pretrain.pt', 
    'lemma_model_path': base + 'en_ewt_lemmatizer.pt', 
    'depparse_model_path': base + 'en_ewt_parser.pt', 
    'depparse_pretrain_path': base + 'en_ewt.pretrain.pt', 
    'tokenize_pretokenized': False # Use pretokenized text as input and disable tokenization
}

nlp = stanfordnlp.Pipeline(**config)

doc = nlp("this is a simple text")
all_information_tab_separated = doc.conll_file.conll_as_string()

parsed_text = "\n".join([f'text: {word.text+" "}\tlemma: {word.lemma}\tupos: {word.upos}\tdepencency: {word.dependency_relation}' for sent in doc.sentences for word in sent.words])

Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': 'C:\\Users\\Marco\\stanfordnlp_resources\\en_ewt_models\\en_ewt_tokenizer.pt', 'pretokenized': False, 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: pos
With settings: 
{'model_path': 'C:\\Users\\Marco\\stanfordnlp_resources\\en_ewt_models\\en_ewt_tagger.pt', 'pretrain_path': 'C:\\Users\\Marco\\stanfordnlp_resources\\en_ewt_models\\en_ewt.pretrain.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: lemma
With settings: 
{'model_path': 'C:\\Users\\Marco\\stanfordnlp_resources\\en_ewt_models\\en_ewt_lemmatizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmatizer with edit classifier]
---
Loading: depparse
With settings: 
{'model_path': 'C:\\Users\\Marco\\stanfordnlp_resources\\en_ewt_models\\en_ewt_parser.pt', 'pretrain_pa

C:\ProgramData\Anaconda3\lib\site-packages\stanfordnlp\models\depparse\model.py:157: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at  ..\aten\src\ATen\native\TensorAdvancedIndexing.cpp:650.)
  unlabeled_scores.masked_fill_(diag, -float('inf'))


In [41]:
print(dir(doc))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_conll_file', '_sentences', '_text', 'conll_file', 'load_annotations', 'sentences', 'text', 'write_conll_to_file']


In [59]:
print(dir(doc.sentences[0].words[0]))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_dependency_relation', '_feats', '_governor', '_index', '_lemma', '_parent_token', '_text', '_upos', '_xpos', 'dependency_relation', 'feats', 'governor', 'index', 'lemma', 'parent_token', 'pos', 'text', 'upos', 'xpos']


In [55]:
doc.sentences[0].words[0].text

'this'

In [57]:
doc.sentences[0].words[1].lemma

'be'

In [67]:
doc.sentences[0].words[2].upos

'DET'

## Functions

In [43]:
# 1
def preprocessing(text):
    text = text.lower() # Lowering case
    remove_url = re.sub(r'(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})', ' ', text) # Removing url
    remove_retweet = re.sub(r"@\w+", " ",remove_url) # Removing retweet
    remove_retweet = re.sub(r"&\w+", " ",remove_retweet) # Remove &amp
    remove_retweet = re.sub(r"\b([!#\$%&\\\(\)\*\+,-\./:;<=>\?@\[\]\^_`\{|\}\"~]+)\b", " ",remove_retweet) # Must check this one
    remove_retweet = re.sub(r"([a-z])\1{3,}", r"\1",remove_retweet)
    remove_punc = remove_retweet.translate(str.maketrans('', '', string.punctuation)) # Remove punctuation
    final_text = re.sub(r'\d+', ' ', remove_punc) # Remove number 
    final_text = re.sub(r'\s+', ' ', final_text) # Removing exceeding spaces
    return final_text

In [5]:
#2
def tokenization(text_clean, tok = "tweet"):
    if tok == "tweet": # TweetTokenizer
        tt = TweetTokenizer()
        tokenized_text = tt.tokenize(text_clean)
    elif tok == "wordpunct": # WordPunctTokenizer
        wpt = WordPunctTokenizer()
        tokenized_text = wpt.tokenize(text_clean)
    return tokenized_text

In [6]:
#3
def remove_stopwords(tokenized_text):
    remove_sw = []
    for token in tokenized_text:
        stop_words.append("rt") # Added a stop words, RT of ReplyTweet I think
        if token.lower() not in stop_words:
             remove_sw.append(token)
    return remove_sw

In [51]:
# 4
# pos-tagging (1 document)
def pos_tagging(doc_token):
    return nltk.pos_tag(doc_token)

# convertion of pos tagging
def get_wordnet_pos(word_tag):
    if word_tag.startswith('J'):
        return "a"
    elif word_tag.startswith('V'):
        return "v"
    elif word_tag.startswith('R'):
        return "r"
    else:
        return "n"
    
# lemmatizer one word 
def lemmatizer(word):
    pos = get_wordnet_pos(word[1])
    wnl = WordNetLemmatizer()
    return wnl.lemmatize(word[0], pos = pos)

# lemmatizer one document
def lemmatizer_doc(doc_token):
    lemmas = [] 
    
    pos_document = pos_tagging(doc_token) # pos tagging
    for token in pos_document:
        lemmas.append( lemmatizer(token) ) # lemmatization x word
    
    return lemmas

In [9]:
# 5
def stemmer(tokenized_text):
    porter = PorterStemmer()
    return [porter.stem(word) for word in tokenized_text]

In [50]:
text = df["tweet"][1432]
print(text)
text_prep = preprocessing(text)
print("1. PREPROCESSING:\n", text_prep)
text_prep = tokenization(text_prep)
print("2. TOKENIZATION:\n", text_prep)
text_prep = remove_stopwords(text_prep)
print("3. STOP WORDS:\n", text_prep)
text_prep = lemmatizer_doc(text_prep)
print("4. LEMMATIZATION:\n", text_prep)
text_prep = stemmer(text_prep)
print("5. STEMMER:\n", text_prep)

&#8220;@Nadixo__: "@joelxmontoya: I'm dreaming &#128525;&#128525; http://t.co/Y21PyZ5rai" ewwww&#8221;bitch you must be fucked up &#128514;&#128514;
1. PREPROCESSING:
  im dreaming ew bitch you must be fucked up 
2. TOKENIZATION:
 ['im', 'dreaming', 'ew', 'bitch', 'you', 'must', 'be', 'fucked', 'up']
3. STOP WORDS:
 ['im', 'dreaming', 'ew', 'bitch', 'must', 'fucked']
4. LEMMATIZATION:
 ['im', 'dream', 'ew', 'bitch', 'must', 'fuck']
5. STEMMER:
 ['im', 'dream', 'ew', 'bitch', 'must', 'fuck']


In [65]:
def processing(text):
    text_prep = preprocessing(text)
    text_prep = tokenization(text_prep)
    text_prep = remove_stopwords(text_prep)
    text_prep = lemmatizer_doc(text_prep)
    #text_prep = stemmer(text_prep)
    print(text_prep)
    return text_prep

In [66]:
df["tweet_clean"] = df["tweet"].apply(lambda x : processing(x))

['woman', 'shouldnt', 'complain', 'cleaning', 'house', 'man', 'always', 'take', 'trash']
['boy', 'dat', 'cold', 'tyga', 'dwn', 'bad', 'cuffin', 'dat', 'hoe', 'st', 'place']
['dawg', 'ever', 'fuck', 'bitch', 'start', 'cry', 'confuse', 'shit']
['look', 'like', 'tranny']
['shit', 'hear', 'might', 'true', 'might', 'faker', 'bitch', 'tell', 'ya']
['shit', 'blow', 'claim', 'faithful', 'somebody', 'still', 'fuck', 'hoe']
['sit', 'hate', 'another', 'bitch', 'get', 'much', 'shit', 'go']
['cause', 'im', 'tire', 'big', 'bitch', 'come', 'u', 'skinny', 'girl']
['might', 'get', 'ya', 'bitch', 'back', 'thats']
['hobby', 'include', 'fight', 'mariam', 'bitch']
['keeks', 'bitch', 'curve', 'everyone', 'lol', 'walk', 'conversation', 'like', 'smh']
['murda', 'gang', 'bitch', 'gang', 'land']
['hoe', 'smoke', 'loser', 'yea', 'go', 'ig']
['bad', 'bitch', 'thing', 'like']
['bitch', 'get']
['bitch', 'nigga', 'miss']
['bitch', 'plz', 'whatever']
['bitch', 'love']
['bitch', 'get', 'cut', 'everyday', 'b']
['black'

['yall', 'still', 'go', 'trick', 'treat', 'year', 'nah', 'hell', 'yeah', 'ima', 'da', 'sweet', 'tooth', 'bandit', 'steal', 'bitch', 'bag', 'n', 'shit']
['wanna', 'sex', 'mom', 'fuck', 'right', 'pussy', 'come', 'damn', 'got', 'hack', 'like', 'fuck']
['kiss', 'u', 'eye', 'open', 'watch', 'bitch', 'lmfao']
['woulda', 'coulda', 'shoulda', 'as', 'hoe']
['happy', 'bday', 'bitch', 'as', 'nigga', 'preciate', 'bitch', 'roll']
['use', 'send', 'bitch']
['ugly', 'bitch', 'like', 'im', 'wcw', 'duh', 'bitch', 'aint', 'nobody', 'el']
['nigga', 'know', 'hacky', 'sack', 'hoe', 'nah', 'throw', 'look', 'pass', 'lmao']
['glad', 'im', 'get', 'outta', 'atlanta', 'nothing', 'bunch', 'nigga', 'outta', 'jail', 'dumb', 'bitch', 'mad', 'kid', 'fuckin', 'em']
['isnt', 'ghetto', 'smart', 'im', 'idc']
['would', 'wife', 'hoe', 'cuz', 'wouldnt', 'care', 'cheat', 'force', 'relationship', 'first', 'place']
['single', 'isnt', 'excuse', 'hoe']
['white', 'boy', 'power', 'bitch', 'black', 'power', 'bitch']
['let', 'get', '

['chill', 'sexy', 'scally', 'lad']
['aint', 'share', 'bitch', 'mine', 'mine']
['money', 'impress', 'simple', 'break', 'bitch']
['pisces', 'never', 'anybodys', 'bitch', 'dont', 'let', 'people', 'control', 'change', 'rigghhht']
['tuku', 'ho', 'idk', 'blah', 'blah', 'blah']
['im', 'mad', 'cuh', 'hahaha', 'suure', 'tuku', 'ho', 'ita']
['hoe', 'crazy']
['aint', 'ever', 'gonna', 'slice', 'bitch', 'kevin', 'hart']
['oomf', 'fuckin', 'fine', 'tho', 'dam', 'dont', 'bitch']
['wonder', 'girl', 'give', 'head', 'boy', 'drink', 'magaluf', 'twitter', 'probly', 'send', 'nude', 'fag']
['newsatquestions', 'yeah', 'bitch', 'yeah', 'bitch', 'call', 'maybe', 'b', 'steve', 'c', 'later', 'jesse', 'pinkman']
['hickies', 'thigh', 'right', 'pussy', 'right']
['would', 'still', 'love', 'im', 'longer', 'young', 'beautiful', 'ugly', 'monkey', 'as']
['yall', 'come', 'front', 'bitch', 'aint', 'come']
['faggot', 'bitch']
['yea', 'im', 'ipad', 'dont', 'work', 'friday', 'let', 'ball', 'faggot', 'work']
['scally', 'lad',

['get', 'couple', 'bitch', 'say', 'know', 'let', 'money', 'talk', 'im', 'like', 'wwyd']
['grudge', 'weirdos', 'bitch']
['tell', 'mariam', 'cigarette', 'birthday', 'also', 'call', 'trash', 'rn']
['brownie', 'tonight', 'kind']
['text', 'back', 'dont', 'give', 'fuck', 'ill', 'text', 'another', 'bitch', 'damn', 'someone', 'hurt', 'lol']
['um', 'okay', 'get', 'hoe', 'leave', 'right', 'yah', 'thang']
['use', 'mx', 'b', 'find', 'slope', 'as']
['fit', 'lad', 'nice', 'gear', 'scally', 'lads', 'n', 'trainer', 'would', 'get']
['hell', 'yeah', 'bro', 'fuck', 'yeah', 'sexy', 'scally', 'lad', 'hot', 'gear']
['help', 'mate', 'love', 'scally', 'play', 'fuck', 'yeah']
['lad', 'lads', 'fuck', 'yeah', 'love', 'scally', 'lad', 'action', 'hot', 'trainer']
['next', 'year', 'bitch', 'gonna', 'like', 'gotta', 'promise', 'wont', 'expose']
['hot', 'lick', 'bitch', 'fuck', 'yeah', 'eat', 'trainer', 'u', 'dirty', 'fucker']
['yall', 'think', 'celebrity', 'pussy', 'different', 'local', 'yeah', 'meghan', 'good']
['s

['im', 'compare', 'kfc', 'pussy']
['im', 'download', 'new', 'personality', 'order', 'new', 'face', 'bitch', 'love', 'goddamn', 'axl', 'rise', 'yell', 'sex', 'robot']
['im', 'feelin', 'like', 'malcolm', 'middle', 'bitch', 'surround', 'cant', 'miss', 'em']
['im', 'gonna', 'piss', 'dimple', 'stop', 'waste', 'prayer', 'trash', 'team', 'lol']
['im', 'probably', 'main', 'bitch', 'chocolate', 'dip', 'cinnamon', 'apple']
['im', 'sick', 'shit', 'man', 'sit', 'bitch', 'as', 'im', 'feed', 'bro', 'geno', 'throw', 'yet', 'another', 'pick', 'smh']
['im', 'sorry', 'fuck', 'yo', 'bitch', 'really', 'dig', 'nigga']
['ibs', 'irritable', 'bitch', 'syndrome']
['love', 'hide', 'yo', 'bitch', 'youll', 'keep', 'shorty']
['talk', 'back', 'aint', 'get', 'pussy']
['youre', 'bad', 'bitch', 'put', 'hand', 'high', 'every', 'girl', 'room', 'start', 'wave', 'hand', 'around']
['imma', 'tie', 'bitch', 'basement', 'donkey', 'fuck', 'as', 'two', 'week', 'straight', 'hahahahahahahahaha', 'centmovie', 'beforeiselfdestruct'

...

['lmao', 'girl', 'dont', 'let', 'guy', 'treat', 'like', 'yellow', 'starburst', 'pink', 'starburst']
['lmao', 'everything', 'bet', 'trash', 'b', 'rad', 'white', 'mike', 'bet', 'cant', 'trash']
['lmao', 'lol', 'ok', 'pal', 'bitch', 'go', 'tell', 'tuh']
['lmao', 'sit', 'ghetto', 'girl', 'seat', 'lunch']
['lmao', 'relax', 'ya', 'mark', 'bitch', 'as', 'roll', 'tide', 'fuck', 'state', 'texas']
['lmao', 'dude', 'say', 'bitch', 'ny', 'fuck', 'redmans', 'great', 'hit']
['lmao', 'eat', 'goal', 'faggot']
['lmfao', 'back', 'dry', 'get', 'shower', 'youre', 'hoe']
['lmfaooo', 'whenever', 'take', 'selfies', 'work', 'always', 'one', 'hatin', 'as', 'bitch', 'comment', 'talmbout', 'aint', 'workin']
['lmfaooo', 'cant', 'yo', 'bitch', 'officially', 'cross', 'line']
['lmfao', 'wiz', 'perfect', 'example', 'fame', 'get', 'bitch', 'look', 'like', 'typical', 'tweaker', 'homeless', 'woman']
['lock', 'bitch', 'keep', 'heat', 'shut', 'front', 'door']
['lol', 'hoe', 'ive', 'realize', 'dont', 'fuckin', 'need']
['lo

['lie', 'tho', 'im', 'weak', 'pretty', 'bitch', 'type', 'shit', 'aint', 'really']
['need', 'bitch']
['paper', 'hoe']
['pill', 'jus', 'weed', 'kill', 'dat', 'pussy', 'night', 'damn', 'right', 'sho', 'u', 'right']
['pillz', 'jus', 'weed', 'kill', 'dat', 'pussy', 'night', 'damn', 'right', 'show', 'u', 'right']
['youre', 'fuck', 'retard', 'thats', 'gonna', 'get', 'fucking', 'date', 'rap', 'frat', 'tonight', 'fun', 'repress', 'memory']
['nobody', 'bout', 'action', 'pussy']
['nobody', 'like', 'bloody', 'nip']
['nobody', 'understand', 'besides', 'day', 'bitch', 'please', 'come', 'back', 'bc', 'im', 'tire', 'female', 'think', 'know']
['nobody', 'want', 'nigga', 'take', 'expensive', 'date', 'dont', 'expect', 'pussy', 'cause', 'take', 'girl', 'like', 'tf']
['none', 'chantel', 'lol', 'get', 'hoes', 'water', 'edge', 'big']
['none', 'bitch', 'look', 'like', 'dey', 'ca', 'boil', 'egg', 'work', 'bud', 'broiler']
['nope', 'order', 'domino', 'trash']
['even', 'sorry', 'lmao', 'grow', 'shit', 'still', '

['shid', 'think', 'legal', 'beat', 'yo', 'hoe']
['shit', 'bitch', 'nigga', 'shit']
['shit', 'wanna', 'know', 'bitch', 'go', 'protect']
['shit', 'piss', 'fuck', 'cunt', 'cocksucker', 'motherfucker', 'tit']
['shit', 'crusty', 'hoe']
['shout', 'pullz', 'cuz', 'kno', 'trash', 'would', 'keep', 'buck', 'swear', 'every', 'make', 'jus', 'cuz', 'n', 'actually', 'bump', 'dem']
['shout', 'bitch', 'like', 'gum', 'chewing', 'shit']
['showin', 'love', 'bitch', 'fuck', 'side']
['shut', 'bitch']
['shutup', 'fag', 'real', 'shit', 'female', 'love', 'real', 'nigga', 'like']
['sick', 'watch', 'yankee', 'game']
['skinny', 'young', 'nigga', 'n', 'im', 'finer', 'bitch']
['slap', 'hoe']
['slap', 'hoe', 'ass', 'like', 'birth', 'em']
['slay', 'lolz', 'jk', 'bitch', 'ill', 'cut', 'emabiggestfans']
['slow', 'reply', 'make', 'feel', 'like', 'retard', 'want', 'put', 'foot', 'wall', 'hjhdyhdhlgjdftilh']
['slut', 'whore', 'hoe']
['slut', 'specky', 'albino', 'lesbo', 'wannabe', 'player', 'dwarf', 'coolcunts', 'absolut

['bitch', 'need', 'call', 'im', 'sorry']
['bitch', 'sing', 'song', 'tho']
['hoe', 'superweak']
['hoe', 'aint', 'loyal', 'n', 'nigga', 'aint', 'either', 'fuck', 'ah', 'bitch', 'get', 'ah', 'bird']
['hoe', 'lie', 'u', 'nigga']
['hoe', 'bout', 'prosper', 'cinegram', 'lordwillin']
['hoe', 'give', 'shame', 'could', 'see', 'cherry', 'stem', 'fuck', 'street']
['ho', 'really', 'hatin', 'draya', 'fine', 'as']
['judge', 'gon', 'try', 'hoe', 'floyd', 'dont', 'show', 'last', 'round']
['niggas', 'bitchin', 'tell', 'em', 'suck', 'sumn', 'bitch', 'see', 'vision', 'tell', 'em', 'fuck', 'sumn']
['pussy', 'as', 'nigga', 'aint', 'bout', 'life']
['blame', 'alchohol', 'claim', 'truth', 'come', 'drunk', 'defend', 'hoe', 'hoe', 'backward', 'as', 'mofos']
['gotta', 'sell', 'ugliness', 'ugly', 'bitch', 'stick', 'like', 'bish']
['make', 'put', 'basketball', 'short', 'look', 'like', 'confused', 'dyke', 'mosh', 'warrior']
['probably', 'hate', 'even', 'look', 'white', 'im', 'technically', 'poc', 'white', 'hair', '

['yo', 'song', 'come', 'hit', 'blunt', 'hard', 'af', 'cause', 'hoe', 'go', 'dat', 'hard']
['first', 'walk', 'door', 'party', 'saw', 'big', 'booty', 'hoe', 'name']
['youre', 'stupid', 'dont', 'know', 'life', 'easy', 'ever', 'see', 'depress', 'retard', 'envy']
['da', 'ratchet', 'hoe']
['scene', 'hoe']
['r', 'fun', 'twin', 'im', 'try', 'lick', 'cheeto', 'dust', 'coleslaw', 'nip']
['whheeet', 'bitch', 'lie']
['one', 'name', 'offensive', 'kike', 'wop', 'kraut', 'wetback', 'jigaboo', 'towelhead', 'gook', 'redskin']
['twitter', 'bitch', 'isnt', 'selling', 'choker']
['whos', 'downie', 'like', 'brownie', 'brunch', 'need', 'mimosa']
['who', 'pussy', 'well', 'crazy', 'bitch']
['people', 'act', 'like', 'didnt', 'know', 'hoe', 'allen', 'go', 'cavs', 'month', 'ago']
['freshman', 'ghetto', 'lmao']
['bitch', 'gotta', 'bitch']
['cant', 'girl', 'admit', 'want', 'dick', 'much', 'u', 'guy', 'want', 'pussy', 'human', 'nature', 'duhh']
['fin', 'sign', 'finnegan', 'he', 'certify', 'bitch']
['hoe', 'think', '

In [72]:
df.to_csv("processing_dataset.csv")

### Interesting rows

- 1432
- 24
- 3129
- 530
- 19999
- 998
- 1605
- 4567
- 555

In [ ]:
base = "C:\\Users\\Marco\\stanfordnlp_resources\\en_ewt_models\\"
config = {
    'processors': 'tokenize,pos,lemma,depparse', # Comma-separated list of processors to use
    'lang': 'en', # Language code for the language to build the Pipeline in
    'model_path': base + 'en_ewt_tokenizer.pt', 
    'pos_model_path': base + 'en_ewt_tagger.pt', 
    'pos_pretrain_path': base + 'en_ewt.pretrain.pt', 
    'lemma_model_path': base + 'en_ewt_lemmatizer.pt', 
    'depparse_model_path': base + 'en_ewt_parser.pt', 
    'depparse_pretrain_path': base + 'en_ewt.pretrain.pt', 
    'tokenize_pretokenized': False # Use pretokenized text as input and disable tokenization
}

nlp = stanfordnlp.Pipeline(**config)

doc = nlp("this is a simple text")
all_information_tab_separated = doc.conll_file.conll_as_string()

parsed_text = "\n".join([f'text: {word.text+" "}\tlemma: {word.lemma}\tupos: {word.upos}\tdepencency: {word.dependency_relation}' for sent in doc.sentences for word in sent.words])
def stanford_parser(text):
    

In [ ]:
# lemmatization ?
# stemming ?
# normalization ?
# n-grams ?

# representation (word2vec - gensim)
# text clustering/classification
# evaluation con confronti
# conclusioni